In [1]:
def init_environment(forzar_drive=False):
  # 1) Montar Google Drive
  from google.colab import drive
  drive.mount('/content/drive', force_remount=forzar_drive)

  # 2) Rutas base
  from pathlib import Path
  BASE_DIR     = Path('/content/drive/MyDrive/structure')
  MODELS_DIR   = BASE_DIR / 'models'

  # 3) Path y sys.path
  import sys
  sys.path.append(str(MODELS_DIR))
  sys.path.append(str(BASE_DIR))

In [ ]:
# =============================================================================
# 🚀 Notebook principal de experimentación con TensorFlow/Keras
# =============================================================================

# 1) Preparar el entorno
#    - Monta Google Drive
#    - Ajusta sys.path para encontrar tus módulos en models/ y utils/
init_environment(forzar_drive=True)

# 2) Definir experimento
#    El nombre debe coincidir con un archivo YAML en configs/experiments/
exp_name = "< EXP >"

# 3) Cargar configuración
from utils.experiment.functions import load_config, load_experiment
cfg = load_config(exp_name)

# 4) ¿Single-split o K-Fold?
k = cfg["dataset"].get("k_folds")

if k is None or k <= 1:

    # —————————————— 4A) Flujo único ——————————————

    #  4A.1) Cargar experimento
    cfg, NNClass, params, dataset, train_data, val_data, val_idx = \
        load_experiment(exp_name)

    #  4A.2) Instanciar y Entrenar
    model   = NNClass(cfg, **params)

    #  4A.3 ) Mostrar resumen de la configuración
    from utils.misc.functions import print_exp_configuration
    print(f"\n✔️ Experimento «{cfg['experiment']['name']}» cargado con éxito.\n")
    print_exp_configuration(cfg)

    #  4A.4) Mostrar arquitectura del modelo
    print("\n📋 Arquitectura del modelo:")
    model.model.summary()

    #  4A.5) Entranamiento (o retomar desde último checkpoint)
    history = model.fit(train_data, val_data)

    #  4A.6) Análisis resultados
    from utils.analysis.analysis import ExperimentAnalyzer
    analyzer = ExperimentAnalyzer(
        model=model.model,
        history=history,
        val_data=val_idx,
        cfg=cfg,
        effects=dataset.get_effects("val"),
        show_plots=True,                    # Mostrar figuras
    )
    analyzer.classification_report()
    analyzer.effect_report()
    analyzer.confusion_matrix(normalize="true")
    analyzer.plot_training_curves()

else:
    # —————————————— 4B) Flujo K-Fold ——————————————

    #  4B.1) Cargar experimento con primer FOLD INDEX (fines informativos en consola)
    cfg, NNClass, params, _, _, _, _ = load_experiment(exp_name, fold_index=0)

    #  4B.2) Instanciar para obtener modelo y poder imprimir parámetros
    model   = NNClass(cfg, **params)

    #  4B.3 ) Mostrar resumen de la configuración
    from utils.misc.functions import print_exp_configuration
    print(f"\n✔️ Experimento «{cfg['experiment']['name']}» cargado con éxito.\n")
    print_exp_configuration(cfg)

    #  4B.4) Mostrar arquitectura del modelo
    print("\n📋 Arquitectura del modelo:")
    model.model.summary()

    #  4B.5) K-FOLD
    for fold in range(k):
        print("\n"*5)
        print(f"\n🔄 Fold {fold+1}/{k}")

        #  4B.5.1) Cargar experimento
        cfg, NNClass, params, dataset, train_data, val_data, val_idx = \
            load_experiment(exp_name, fold_index=fold)

        #  4B.5.2) Instanciar y Entrenar
        model   = NNClass(cfg, **params)
        history = model.fit(train_data, val_data)

        #  4B.5.3) Análisis resultados individual
        from utils.analysis.analysis import ExperimentAnalyzer
        analyzer = ExperimentAnalyzer(
            model=model.model,
            history=history,
            val_data=val_idx,
            cfg=cfg,
            fold_index=fold,
            effects=dataset.get_effects("val"),
            show_plots=False,                    # No mostrar figuras individuales de cada k-fold
        )
        
        # Guardar métricas en JSON
        analyzer.classification_report()
        analyzer.effect_report()
        analyzer.confusion_matrix(normalize="true")

    #  4B.6) Análisis resultados de k-fold agrupado
    from utils.analysis.analysis_kfold import ExperimentKFoldAnalyzer
    kf_analyzer = ExperimentKFoldAnalyzer(
        cfg=load_config(exp_name)
        )
    kf_analyzer.report_summary()
    kf_analyzer.plot_evaluation()
    kf_analyzer.plot_accuracy_summary(confidence=0.95)

print("\n"*4)
print("="*15, " | PROCESO FINALIZADO CORRECTAMENTE | ", "="*15)
print("\n"*4)